In [367]:
import requests
from bs4 import BeautifulSoup as soup
import os, time
import json,re
from googletrans import Translator
from google.transliteration import transliterate_word

In [368]:
filmlk_url = 'https://www.films.lk/'

In [369]:
filename = "artists-corpus/artists_links.csv"

def scape_main_pages():
    with open(filename,'w') as f:
        for i in range(1,30):
            main_url = 'https://www.films.lk/artists.php?page={}'.format(i)
            print("scrapping : " + main_url)
            response = requests.get(
                url = main_url,
            )

            page_soup = soup(response.content, 'html.parser')
            artists = page_soup.findAll("div",{"class":"column2"})


            for artist in artists:
                name = artist.span.text
                url = artist.a["href"]
                f.write(url + "\n")

            time.sleep(10)

In [7]:
scape_main_pages()

scrapping : https://www.films.lk/artists.php?page=1
scrapping : https://www.films.lk/artists.php?page=2
scrapping : https://www.films.lk/artists.php?page=3
scrapping : https://www.films.lk/artists.php?page=4
scrapping : https://www.films.lk/artists.php?page=5
scrapping : https://www.films.lk/artists.php?page=6
scrapping : https://www.films.lk/artists.php?page=7
scrapping : https://www.films.lk/artists.php?page=8
scrapping : https://www.films.lk/artists.php?page=9
scrapping : https://www.films.lk/artists.php?page=10
scrapping : https://www.films.lk/artists.php?page=11
scrapping : https://www.films.lk/artists.php?page=12
scrapping : https://www.films.lk/artists.php?page=13
scrapping : https://www.films.lk/artists.php?page=14
scrapping : https://www.films.lk/artists.php?page=15
scrapping : https://www.films.lk/artists.php?page=16
scrapping : https://www.films.lk/artists.php?page=17
scrapping : https://www.films.lk/artists.php?page=18
scrapping : https://www.films.lk/artists.php?page=19
sc

In [404]:
def get_artist_list(start, end): 
    with open(filename, "r") as f:
        artists_urls = f.readlines()
    
    artist_list = []
    
    for i in range(start, end):
        response = requests.get(
            url = filmlk_url + artists_urls[i].rstrip('\n'),
        )
  
        artist_data = get_artist_meta_data(response.content) 
    
        # biography
        artist_bio_data = get_artist_bio_data(response.content) 
        artist_data['biography_en'] = artist_bio_data
        
        # filmography
        artist_film_data = get_artist_film_data(response.content)  
        
        if (len(artist_film_data) < 10):
            continue
        
        artist_data['filmography_en'] = artist_film_data
        
        with open ('artists-corpus/artists_data.json','r') as f:
            data = json.load(f)
        
        data.append(artist_data)
        
        with open('artists-corpus/artists_data.json', "w") as f:
            json.dump(data, f)
                  
        artist_list.append(artist_data)
          
        time.sleep(5)
        
    return artist_list            

In [405]:
def get_artist_meta_data(content):
    
    page_soup = soup(content, 'html.parser')
    artist = page_soup.findAll("div",{"class":"column2"})
    artist_summary = artist[1].findAll("li",{"class":None})
        
    artist_meta_data = {}
    artist_meta_data["known_as_en"] = ""
    artist_meta_data["known_as_si"] = ""
    artist_meta_data["real_name_en"] = ""
    artist_meta_data["birth_en"] = ""
    artist_meta_data["death_en"] = ""
    
    for field in artist_summary:
        if field.b != None:

            if "Known As" in field.b.get_text():
                known_as_en = field.text.split(":")[1].strip()
                print("scrapping : " + known_as_en)
                artist_meta_data["known_as_en"] = known_as_en

            if "Sinhala" in field.b.get_text(): 
                known_as_si = field.text.split(":")[1].strip()
                artist_meta_data["known_as_si"] = known_as_si

            if "Real Name" in field.b.get_text(): 
                real_name = field.text.split(":")[1].strip()
                artist_meta_data["real_name_en"] = real_name

            if "Birthday" in field.b.get_text(): 
                birth = field.text.split(":")[1].strip() 
                artist_meta_data["birth_en"] = birth

            if "Died" in field.b.get_text(): 
                death = field.text.split(":")[1].strip()
                artist_meta_data["death_en"] = death
    
    return artist_meta_data
    

In [406]:
def get_artist_film_data(content):
    
    page_soup = soup(content, 'html.parser')
    columns = page_soup.findAll("div",{"class":"column"})
    artist_film_data = []
    
    for column in columns:
        field = column.find("span",{"class":"eventTitle"})   

        if (field != None and field.get_text() == 'FILMOGRAPHY'): 

            titles = (column.findAll("span",{"class":"eventTitle"}))

            titles = list(map(lambda title: title.get_text(), titles))

            if 'AS CAST MEMBER' not in titles:
                continue

            td_rows = column.table.findAll("td",{"class":"row1"})

            for row in td_rows:

                if (row.span != None and row.a != None):
                    filmography = {}

                    role_name = row.span.text
                    film_name = row.a.text

                    filmography["film_name_en"] = film_name
                    filmography["role_name_en"] = role_name

                    artist_film_data.append(filmography) 
                    
    return artist_film_data

In [407]:
def get_artist_bio_data(content):
    page_soup = soup(content, 'html.parser')
    columns = page_soup.findAll("div",{"class":"column"})
    artist_bio_data = ""
    
    for column in columns:
        field = column.find("span",{"class":"eventTitle"})   

        if (field != None and field.get_text() == 'BIOGRAPHY'): 

            for p in column.findAll('p',{"class":None}):
                artist_bio_data += (p.text)
                
    return artist_bio_data                

In [411]:
artist_list = get_artist_list(120,200)
artist_list

scrapping : Quintus Weerakoon
scrapping : Robin Fernando
scrapping : Rodney Warnakula
scrapping : Roger Senewirathna
scrapping : Ronnie Leitch
scrapping : Roshan Pilapitiya
scrapping : Sahan Wijesinghe
scrapping : Sando Haris
scrapping : Sandun Wijesiri
scrapping : Sanjaya Leelarathna
scrapping : Sarath Dikkumbura
scrapping : Sarath Kothalawala
scrapping : Sathischandra Edirisinghe
scrapping : Saumya Liyanage
scrapping : Shesha Palihakkara
scrapping : Stanley Perera
scrapping : Sunil Hettiarachchi
scrapping : T.M. Sangadasa
scrapping : Teddy Vidyalankara
scrapping : Tissa Wijesundara
scrapping : Thilak Kumara Rathnayaka
scrapping : Vijaya Nandasiri
scrapping : W. Jayasiri
scrapping : Wasantha Kumarawila
scrapping : Wasantha Wittachchi
scrapping : Don Sirisena
scrapping : Tissa Abeysekara
scrapping : Nirosha Perera
scrapping : Chathurika Pieris
scrapping : Oshadie Hewamadduma
scrapping : Udula Dabare
scrapping : Senadeera Kuruppu
scrapping : Ravindra Randeniya
scrapping : Sriyani Amaras

[{'known_as_en': 'Quintus Weerakoon',
  'known_as_si': '',
  'real_name_en': 'Quintus Weerakoon',
  'birth_en': 'April 20',
  'death_en': '',
  'biography_en': '',
  'filmography_en': [{'film_name_en': 'Kathuru Muwath',
    'role_name_en': 'Actor'},
   {'film_name_en': 'Mathara Achchi', 'role_name_en': 'Actor'},
   {'film_name_en': 'Thilaka Ha Thilakaa', 'role_name_en': 'Actor'},
   {'film_name_en': 'Sudu Paraviyo', 'role_name_en': 'Actor'},
   {'film_name_en': 'Nivena Ginna', 'role_name_en': 'Actor'},
   {'film_name_en': 'Sasara', 'role_name_en': 'Actor'},
   {'film_name_en': 'Rosa Mal Thunak', 'role_name_en': 'Actor'},
   {'film_name_en': 'Anusha', 'role_name_en': 'Actor'},
   {'film_name_en': 'Jodu Valalu', 'role_name_en': 'Actor'},
   {'film_name_en': 'Eka Dawasak Re', 'role_name_en': 'Actor'},
   {'film_name_en': 'Chandi Patavu', 'role_name_en': 'Actor'},
   {'film_name_en': 'Hitha Honda Kollek', 'role_name_en': 'Actor'},
   {'film_name_en': 'Hithawathiya', 'role_name_en': 'Actor'